In [1]:
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
from typing import Iterable
import padding as P

import torch
from torch import nn
import torch.nn.functional as F
from torch import optim
from torch.utils import data

In [2]:
bsz, channels, depth, height, width = 4, 3, 5, 240, 240
input_size = (bsz, channels, depth, height, width)
X = torch.rand(input_size)
print(X.shape, X.max(), X.min())

torch.Size([4, 3, 5, 240, 240]) tensor(1.0000) tensor(5.9605e-07)


In [3]:
_kernel_size = (3, 3, 3)
_stride = (1, 2, 3)
_dilation = (1, 1, 1)
_padding = (1, 1, 1)

pad = P.Pad3D(padding=_padding)
padded_X = pad(X)
print(padded_X.shape)

torch.Size([4, 3, 7, 242, 242])
